In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [3]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [5]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [6]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
1013,01/12/2020 08:00:20,32.0,Male,Munich,Product Manager,4,4,Senior,NaN,"Java / Scala, SQL",65000.0,65000,30,Full-time employee,Unlimited contract,English,1000+,Consulting / Agency
582,25/11/2020 13:43:52,30.0,Male,Berlin,Frontend Developer,4,2,Senior,TypeScript,Javascript / Typescript,73000.0,0,26,Full-time employee,Unlimited contract,English,101-1000,Product
87,24/11/2020 11:46:15,32.0,Female,Berlin,QA Engineer,10,NaN,Senior,Java,"Kotlin, Javascript / Typescript, SQL, Docker",61000.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product
974,30/11/2020 16:20:18,32.0,Female,Berlin,Engineering Team Lead,10,6,Lead,QA,NaN,65000.0,3250,28,Full-time employee,Unlimited contract,English,101-1000,Product
1188,22/12/2020 13:39:31,36.0,Male,Berlin,Backend Developer,12,0.5,Senior,Python,"SQL, Docker",100000.0,15000,30,Full-time employee,Unlimited contract,English,1000+,Startup
280,24/11/2020 15:07:35,33.0,Male,Frankfurt,Software Engineer,10,1,Senior,.NET,"Javascript / Typescript, SQL, Azure",26400.0,NaN,NaN,Self-employed (freelancer),Unlimited contract,English,up to 10,Startup
705,25/11/2020 21:48:49,29.0,Male,Berlin,Mobile Developer,5,4,Senior,iOS,Swift,65000.0,NaN,27,Full-time employee,Unlimited contract,English,101-1000,Product
360,24/11/2020 18:16:39,25.0,Male,Berlin,Backend Developer,5,0,Senior,Java,Python,60000.0,5600,NaN,NaN,NaN,NaN,NaN,NaN
1166,16/12/2020 12:02:55,27.0,Male,Berlin,Data Engineer,0,0,Junior,Python,"Python, SQL, Azure, Docker",40000.0,0,30,Full-time employee,Unlimited contract,German,1000+,Consulting / Agency
152,24/11/2020 12:14:03,30.0,Male,Berlin,Product Manager,7,5,Head,NaN,"Python, SQL",150000.0,NaN,25,Full-time employee,Unlimited contract,English,1000+,Startup


In [7]:
# Part 1: Data Cleaning and Exploration

# Load Data
df = pd.read_csv("data/Euro_Salary.csv")

# Construct Total Compensation
df['Yearly brutto salary (without bonus and stocks) in EUR'] = pd.to_numeric(df['Yearly brutto salary (without bonus and stocks) in EUR'], errors='coerce')
df['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors='coerce')

# Handling NaN values after conversion
df['Yearly brutto salary (without bonus and stocks) in EUR'].fillna(0, inplace=True)
df['Yearly bonus + stocks in EUR'].fillna(0, inplace=True)

df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

# Explore Data
print(df.describe(include="all").T)

# Check for missing values
print(df.isnull().sum())

# Display a subset of the data
print(df[393:396])
print(df.info())
print(df["Other technologies/programming languages you use often"].value_counts())



                                                     count unique  \
Timestamp                                             1253   1248   
Age                                                 1226.0    NaN   
Gender                                                1243      3   
City                                                  1253    119   
Position                                              1247    148   
Total years of experience                             1237     48   
Years of experience in Germany                        1221     53   
Seniority level                                       1241     24   
Your main technology / programming language           1126    256   
Other technologies/programming languages you us...    1096    562   
Yearly brutto salary (without bonus and stocks)...  1253.0    NaN   
Yearly bonus + stocks in EUR                        1253.0    NaN   
Number of vacation days                               1185     45   
Employment status                 

In this section, we load the dataset and construct the "Total Compensation" variable by combining yearly salary and bonus/stocks. We handle NaN values and ensure numeric consistency. The exploration includes descriptive statistics, checking for missing values, and displaying a subset of the data. Further insights into the dataset structure are gained through info and value counts. This initial exploration sets the stage for subsequent analysis and modeling steps. 

In [8]:
#Dataset information and preview 
# Display basic information about the dataset
print(df.info())

# Display the first few rows of the dataset
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 19 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Timestamp                                               1253 non-null   object 
 1   Age                                                     1226 non-null   float64
 2   Gender                                                  1243 non-null   object 
 3   City                                                    1253 non-null   object 
 4   Position                                                1247 non-null   object 
 5   Total years of experience                               1237 non-null   object 
 6   Years of experience in Germany                          1221 non-null   object 
 7   Seniority level                                         1241 non-null   object 
 8   Your main technology / programming lan

The code snippet provides essential information about the dataset using df.info(), offering insights into the data types, non-null counts, and memory usage. This concise summary aids in understanding the dataset's structure and potential data type issues.
The subsequent use of df.head() displays the first few rows of the dataset, allowing for a quick preview of the actual data. This preview assists in identifying the column names, initial values, and overall data format.

In [9]:
#DATA PREPROCESSING
# Drop unnecessary columns
df = df.drop(columns=['Timestamp'])

# Handling extreme values in 'Yearly brutto salary (without bonus and stocks) in EUR'
salary_threshold = 500000
df['Yearly brutto salary (without bonus and stocks) in EUR'] = np.where(
    df['Yearly brutto salary (without bonus and stocks) in EUR'] > salary_threshold,
    np.nan,
    df['Yearly brutto salary (without bonus and stocks) in EUR']
)

# Handling missing values
df = df.dropna()

# Convert categorical variables to numerical using one-hot encoding
df = pd.get_dummies(df, drop_first=True)

# Split the data into features (X) and target variable (y)
X = df.drop(columns=['Yearly bonus + stocks in EUR'])
y = df['Yearly bonus + stocks in EUR']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In this section, several preprocessing steps are performed to enhance the dataset's suitability for regression modeling. Initially, unnecessary columns, such as 'Timestamp,' are dropped to focus on relevant features. Extreme values in the 'Yearly brutto salary' column are addressed by replacing values exceeding a defined threshold with NaN. Rows containing missing values are then dropped. Categorical variables are transformed into numerical format through one-hot encoding. The dataset is split into features (X) and the target variable (y), followed by a division into training and testing sets. Lastly, numerical features are standardized using Min-Max scaling, ensuring a consistent scale across the dataset. These preprocessing steps lay the foundation for building and evaluating regression models.

In [10]:
#Coloumn Processing 
# Specify the column name
tech_column_name = "Other technologies/programming languages you use often"

# Check if the specified column name is in the DataFrame
if tech_column_name not in df.columns:
    print(f"Error: Column '{tech_column_name}' not found in the DataFrame.")
else:
    # Extracting unique technologies/programming languages
    all_technologies = set()
    for tech_list in df[tech_column_name].str.split(', '):
        if isinstance(tech_list, list):
            all_technologies.update(tech_list)

    # Check if there are any unique technologies
    if not all_technologies:
        print(f"Error: No unique technologies found in the column '{tech_column_name}'.")
    else:
        # Creating binary features for each unique technology
        for tech in all_technologies:
            df[tech] = df[tech_column_name].str.contains(tech).astype(int)

        # Drop the original column
        df = df.drop(columns=[tech_column_name])



Error: Column 'Other technologies/programming languages you use often' not found in the DataFrame.


This code segment addresses the specified technology column in the dataset. It first checks if the specified column name exists in the DataFrame, providing an error message if not found. If the column exists, the code extracts unique technologies/programming languages from the specified column, creating binary features for each unique technology through one-hot encoding. The original technology column is then dropped to maintain a more structured and feature-rich dataset for regression analysis. This step facilitates incorporating categorical information into the model and enhances its predictive capabilities.

In [11]:
# Display the list of available columns
print("Available Columns:", df.columns)


Available Columns: Index(['Age', 'Yearly brutto salary (without bonus and stocks) in EUR',
       'Yearly bonus + stocks in EUR', 'Total Compensation', 'Gender_Female',
       'Gender_Male', 'City_Amsterdam', 'City_Ansbach', 'City_Barcelona',
       'City_Berlin',
       ...
       'Company type_Semiconductor', 'Company type_Startup',
       'Company type_Telecommunications', 'Company type_Transport & Logistics',
       'Company type_University', 'Company type_Utilities',
       'Company type_corporate incubator', 'Company type_e-commerce',
       'Company type_eCommerce', 'Company type_service'],
      dtype='object', length=1117)


In [12]:
#Model Pipeline
# Separate features (X) and target variable (y)
X = df.drop(columns=['Total Compensation'])
y = df['Total Compensation']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline for preprocessing and modeling
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # You can replace MinMaxScaler with other scalers if needed
    ('regressor', LinearRegression())  # You can replace LinearRegression with your chosen regressor
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', LinearRegression())])

In this section, the dataset is prepared for regression modeling. Features (X) and the target variable (y) are separated, and the data is split into training and testing sets. A pipeline is then created to streamline the preprocessing and modeling steps. The pipeline includes scaling numerical features using Min-Max scaling and employs a linear regression model as the chosen regressor. The pipeline is fitted to the training data, allowing for a seamless integration of preprocessing techniques and model training. This organized approach enhances code readability and ensures consistency in applying transformations across training and testing datasets.

In [13]:
#Model evaluation 
# Predict on the training set
y_train_pred = pipeline.predict(X_train)

# Predict on the testing set
y_test_pred = pipeline.predict(X_test)

# Evaluate the model
train_r2 = pipeline.score(X_train, y_train)
test_r2 = pipeline.score(X_test, y_test)

train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

# Display the results
print("Training R2:", train_r2)
print("Testing R2:", test_r2)
print("Training RMSE:", train_rmse)
print("Testing RMSE:", test_rmse)


Training R2: 1.0
Testing R2: 0.9981921571377701
Training RMSE: 2.1576749471698455e-10
Testing RMSE: 1866.4261302207183


This section involves predicting the total compensation for both the training and testing sets using the trained regression model. The model's performance is then assessed by calculating the R2 score and root mean squared error (RMSE) for both the training and testing datasets. R2 score indicates the proportion of variance in the target variable that the model explains, while RMSE quantifies the prediction error. These metrics offer insights into the model's accuracy and generalization to unseen data. The displayed results provide a comprehensive overview of the model's performance on both the training and testing datasets, aiding in the assessment of its predictive capabilities.

In [14]:
#Feature selection with Lasso regression 
# Perform feature selection (example using Lasso regression)
lasso_model = Lasso(alpha=0.01)  # You can adjust the alpha value
lasso_model.fit(X_train, y_train)

selected_features = X_train.columns[lasso_model.coef_ != 0]

# Update X_train and X_test with selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Fit the model with selected features
pipeline.fit(X_train_selected, y_train)

# Evaluate the model with selected features
selected_train_r2 = pipeline.score(X_train_selected, y_train)
selected_test_r2 = pipeline.score(X_test_selected, y_test)

selected_train_rmse = mean_squared_error(y_train, pipeline.predict(X_train_selected), squared=False)
selected_test_rmse = mean_squared_error(y_test, pipeline.predict(X_test_selected), squared=False)

# Display the results after feature selection
print("Training R2 (after feature selection):", selected_train_r2)
print("Testing R2 (after feature selection):", selected_test_r2)
print("Training RMSE (after feature selection):", selected_train_rmse)
print("Testing RMSE (after feature selection):", selected_test_rmse)


Training R2 (after feature selection): 1.0
Testing R2 (after feature selection): 1.0
Training RMSE (after feature selection): 1.5335166690164888e-11
Testing RMSE (after feature selection): 1.5006846805699433e-11


n this section, feature selection is performed using Lasso regression. The Lasso model is trained on the training data with a specified alpha value, which controls the strength of regularization. Features with non-zero coefficients in the Lasso model are considered important and selected for further analysis. The chosen features are then used to update both the training and testing datasets. The regression model is refit using the selected features, and its performance is evaluated on both the training and testing sets. The displayed results include the R2 score and root mean squared error (RMSE) after feature selection, providing insights into the impact of feature selection on the model's predictive capabilities. Adjusting the alpha value allows for tuning the level of regularization and exploring different sets of selected features.

# Answers and Explainations
(Expand/modify as needed)

### Results
Training R2: The coefficient of determination (R2) for the training set measures the proportion of the response variable variance captured by the model. A higher R2 indicates better model performance on the training data.
Testing R2: Similar to the training R2, this metric evaluates the model's ability to generalize to unseen data.
Training RMSE: The root mean squared error (RMSE) on the training set represents the average prediction error. Lower values indicate better model performance.
Testing RMSE: RMSE for the testing set provides insights into the model's predictive accuracy on new, unseen data.
### Feature Selection Activities
Lasso Regression:
Lasso regression is employed for feature selection, where the alpha parameter controls the strength of regularization. A non-zero coefficient in the Lasso model indicates the importance of a feature.
Selected features are then used to retrain the model and evaluate its performance

### Hyperparameter Changes
Alpha Tuning in Lasso Regression:
The alpha parameter in Lasso regression is a hyperparameter controlling the level of regularization. Different alpha values can be explored to find the optimal balance between feature selection and model performance.
Adjusting the alpha allows fine-tuning the model's sensitivity to the importance of features. Higher alpha values result in sparser feature selection.